In [6]:
import pandas as pd

file_path = 'C:/Users/moura/PAD_JOB/data_extracted/New-dataset-SVCpredictions.csv'
dataset = pd.read_csv(file_path)


dataset.head()

,title,text,sentiment,stars,likesCount,reviewerPhotoUrl,publishedAtDate,name,predictions
0,BOB'S,Shake e Sunday aguados. Horrível. Não recomendo.,negative,1,1,https://lh3.googleusercontent.com/a-/AD_cMMRso...,2021-04-03 20:10:01.702000+00:00,Sílvio Brito,negative
1,BOB'S,Credo!,negative,1,0,https://lh3.googleusercontent.com/a-/AD_cMMS7i...,2019-05-25 21:11:05.280000+00:00,Tiago de Bem,negative
2,BOB'S,"Desorganizado, pedido veio errado",negative,1,0,https://lh3.googleusercontent.com/a-/AD_cMMQP2...,2020-02-15 23:29:17.166000+00:00,paulo aecio alves,negative
3,BOB'S,Atendimento péssimo,negative,1,0,https://lh3.googleusercontent.com/a-/AD_cMMSwu...,2018-10-18 10:25:35.416000+00:00,Eryka Ferreira Menezes de Paula,negative
4,BOB'S,"Todos atendentes de mal humor e sem educação, ...",negative,1,2,https://lh3.googleusercontent.com/a-/AD_cMMS_z...,2022-01-15 18:37:15.511000+00:00,João Benuto,negative


In [7]:
dataset.iloc[2].at['text']

'Desorganizado, pedido veio errado'

In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89212 entries, 0 to 89211
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             89212 non-null  object
 1   text              89212 non-null  object
 2   sentiment         89212 non-null  object
 3   stars             89212 non-null  int64 
 4   likesCount        89212 non-null  int64 
 5   reviewerPhotoUrl  89212 non-null  object
 6   publishedAtDate   89212 non-null  object
 7   name              89212 non-null  object
 8   predictions       89212 non-null  object
dtypes: int64(2), object(7)
memory usage: 6.1+ MB


In [10]:
import pandas as pd

def map_sentiment(totalScore):
    if totalScore <= 2:
        return "negativo"
    elif totalScore == 3:
        return "neutro"
    else:
        return "positivo"
    
sentiment = dataset['totalScore'].apply(map_sentiment)


dataset = pd.DataFrame({
    'title' : dataset['title'],
    'categoryName': dataset['categoryName'],
    'postalCode' : dataset['postalCode'],
    'address' : dataset['address'],
    'reviews': dataset['reviews'],  
    'SENTIMENTO': sentiment,
    'totalScore': dataset['totalScore'],
    'texto': dataset['texto']
    #'TextBlob': dataset['TextBlob'],    
    #'texto_traduzido': dataset['texto_traduzido']
})



Análise de Sentimentos por Comércio


In [19]:
grouped = dataset.groupby('title').agg(
    #contando quantas vezes title ocorreu
    count_titulos=('title', 'count'),
    #soma das estrelas
    sum_estrelas=('stars', 'sum')
).reset_index()
print(grouped)




                                                 title  count_titulos  \
0                          20 Buscar Bar e Restaurante            184   
1    597 Burger (Hambúrguer, Batata frita, Cerveja)...            407   
2                                          6 Pira Açai             10   
3                         A GRANDE FAMÍLIA RESTAURANTE             22   
4                           ABSOLUTO BAR & RESTAURANTE            253   
..                                                 ...            ...   
843                                       brasa burg's              7   
844                                      buchecha burg              1   
845                     restaurante Caseiro casa de vó              7   
846                         tioBák - Passeio das Àguas            106   
847                               top grill costelaria              8   

     sum_estrelas  
0             677  
1            1807  
2              49  
3             101  
4            1224  
.. 

In [21]:
grouped = dataset.groupby('titulos').agg(
    count_titulos=('titulos', 'count'),
    sum_estrelas=('stars', 'sum')
).reset_index()

grouped['media_estrelas'] = grouped['stars'] / grouped['count_titulos']


KeyError: 'titulos'

In [24]:
#fazern uma restrição para pegar apenas mais de 10 avaliações
analise_comercio = dataset.groupby('title')[['stars']].mean()

# Ordenando os comércios pela média da avaliação
analise_comercio.sort_values('stars', ascending=False).head(10)


,stars
title,
NA CHAPA QUENTE,5.0
Restaurante bar do Léo,5.0
Chicão - Rei Do Frango Assado (Goiânia),5.0
Restaurante e Empório Zion,5.0
Chopp Da Dona Ana,5.0
Restaurante Chickeria Frango Mais,5.0
Restaurante doce Pimenta,5.0
Junior's Hamburgueria,5.0
Jantinha Águia,5.0


In [12]:
# Agrupando o dataset por comércio e calculando a média da avaliação
analise_comercio_avaliacao = dataset.groupby('title')['stars'].mean().reset_index()

# Ordenando os comércios pela média da avaliação
top_comercios = analise_comercio_avaliacao.sort_values('stars', ascending=False).head(5)
bottom_comercios = analise_comercio_avaliacao.sort_values('stars').head(5)

#top_comercios, bottom_comercios


In [13]:
print(top_comercios)

                                       title  stars
371                          NA CHAPA QUENTE    5.0
673                   Restaurante bar do Léo    5.0
134  Chicão - Rei Do Frango Assado (Goiânia)    5.0
698               Restaurante e Empório Zion    5.0
136                        Chopp Da Dona Ana    5.0


In [14]:
print(bottom_comercios)

                            title  stars
373         Na Brasa Hamburgueria    1.0
660     Restaurante Tepero de Mãe    1.0
177            Dias frango Assado    1.0
672           Restaurante avenida    1.0
467  Quiosque Bob's Cidade Jardim    1.0


In [21]:
# Selecionando comentários para o comércio com a maior avaliação média
comercio_maior_avaliacao = top_comercios.iloc[0]['categoryName']

comentarios_maior_avaliacao = dataset[dataset['categoryName'] == comercio_maior_avaliacao][['texto', 'totalScore']]



sample = comentarios_maior_avaliacao.sample(5, replace=True)
print(sample)

                                    texto  totalScore
32  A melhor boate da região.super indico         5.0
33       Super indico em qualquer ocasião         5.0
32  A melhor boate da região.super indico         5.0
32  A melhor boate da região.super indico         5.0
32  A melhor boate da região.super indico         5.0
